---

https://github.com/meta-llama/llama-recipes/blob/v0.0.3/recipes/quickstart/finetuning/quickstart_peft_finetuning.ipynb 를 복사해서 수정함.

pip install llama-recipes 실행 시, 0.0.3 버전이 설치 되기 때문에,
quickstart_peft_finetuning.ipynb 파일도 0.0.3 버전을 사용해야 문제 없이 실행이 됨!

---

Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/finetuning/quickstart_peft_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PEFT Finetuning Quick Start Notebook

This notebook shows how to train a Meta Llama 3 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA finetuning.

**_Note:_** To run this notebook on a machine with less than 24GB VRAM (e.g. T4 with 16GB) the context length of the training dataset needs to be adapted.
We do this based on the available VRAM during execution.
If you run into OOM issues try to further lower the value of train_config.context_length.

### Step 0: Install pre-requirements and convert checkpoint

We need to have llama-recipes and its dependencies installed for this notebook. Additionally, we need to log in with the huggingface_cli and make sure that the account is able to to access the Meta Llama weights.

In [1]:
# uncomment if running from Colab T4
# ! pip install llama-recipes ipywidgets

# import huggingface_hub
# huggingface_hub.login()

### Step 1: Load the model

Setup training configuration and load the model and tokenizer.

In [6]:
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from llama_recipes.configs import train_config as TRAIN_CONFIG

train_config = TRAIN_CONFIG()
train_config.model_name = "meta-llama/Meta-Llama-3.1-8B"
train_config.num_epochs = 1
train_config.run_validation = False
train_config.gradient_accumulation_steps = 4
train_config.batch_size_training = 1
train_config.lr = 3e-4
train_config.use_fast_kernels = True
train_config.use_fp16 = True
train_config.context_length = 1024 if torch.cuda.get_device_properties(0).total_memory < 16e9 else 2048 # T4 16GB or A10 24GB
train_config.batching_strategy = "packing"
train_config.output_dir = "meta-llama-samsum"

from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = LlamaForCausalLM.from_pretrained(
            train_config.model_name,
            device_map="auto",
            quantization_config=config,
            use_cache=False,
            attn_implementation="sdpa" if train_config.use_fast_kernels else None,
            torch_dtype=torch.float16,
        )

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Step 2: Check base model

Run the base model on an example input:

In [7]:
eval_prompt = """
Summarize this dialog:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    generated = model.generate(**model_input, max_new_tokens=100)
    print(tokenizer.decode(generated[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this dialog:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
---
Summary:
#Person1#: Hello, Mr. Smith. I'm Dr. Hawkins. Why did you come today?
#Person2#: I thought I should get a checkup.
#Person1#: You haven't had one in 5 years. You should have one every year.
#Person2#: I know. But why should I see a doctor if I'm healthy?
#Person1#: The best way to prevent serious illness is to detect it early. So, for your health, you should


We can see that the base model only repeats the conversation.

### Step 3: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

In [6]:
from torch.utils.data import Dataset
import pandas as pd

class DatasetForLlamaTrain(Dataset):
    def __init__(self, train_csv_fullpath):
        df = pd.read_csv(train_csv_fullpath)
        
        prompt = (
            f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n"
        )

        def apply_prompt_template(s):
            return prompt.format(dialog=s)

        df['dialogue'] = df['dialogue'].map(apply_prompt_template)

        self.preprocessed_list = []
        
        for i in range(len(df)):
            processed_row = {}
            
            prompt = tokenizer.encode(tokenizer.bos_token + df.iloc[i]['dialogue'], add_special_tokens=False)
            summary = tokenizer.encode(df.iloc[i]['summary'] +  tokenizer.eos_token, add_special_tokens=False)
            
            processed_row = {
                "input_ids": prompt + summary,
                "attention_mask" : [1] * (len(prompt) + len(summary)),
                "labels": [-100] * len(prompt) + summary,
            }
            
            self.preprocessed_list.append(processed_row)
        
        self.len = len(self.preprocessed_list)

    def __getitem__(self, idx):
        return self.preprocessed_list[idx]

    def __len__(self):
        return self.len   

In [7]:
#from llama_recipes.configs.datasets import samsum_dataset
from llama_recipes.data.concatenator import ConcatDataset
from llama_recipes.utils.config_utils import get_dataloader_kwargs
#from llama_recipes.utils.dataset_utils import get_preprocessed_dataset

train_dataset = DatasetForLlamaTrain('../../data/train.csv')

train_dl_kwargs = get_dataloader_kwargs(train_config, train_dataset, tokenizer, "train")

if train_config.batching_strategy == "packing":
        train_dataset = ConcatDataset(train_dataset, chunk_size=train_config.context_length)

# Create DataLoaders for the training and validation dataset
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    **train_dl_kwargs,
)

Preprocessing dataset: 100%|██████████| 12457/12457 [00:00<00:00, 43242.11it/s]


### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [3]:
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from dataclasses import asdict
from llama_recipes.configs import lora_config as LORA_CONFIG

lora_config = LORA_CONFIG()
lora_config.r = 8
lora_config.lora_alpha = 32
lora_dropout: float=0.01

peft_config = LoraConfig(**asdict(lora_config))

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Step 5: Fine tune the model

Here, we fine tune the model for a single epoch.

In [9]:
import torch.optim as optim
from llama_recipes.utils.train_utils import train
from torch.optim.lr_scheduler import StepLR

model.train()

optimizer = optim.AdamW(
            model.parameters(),
            lr=train_config.lr,
            weight_decay=train_config.weight_decay,
        )
scheduler = StepLR(optimizer, step_size=1, gamma=train_config.gamma)

# Start the training process
results = train(
    model,
    train_dataloader,
    None,
    tokenizer,
    optimizer,
    scheduler,
    train_config.gradient_accumulation_steps,
    train_config,
    None,
    None,
    None,
    wandb_run=None,
)

/opt/conda/lib/python3.10/site-packages/llama_recipes/utils/train_utils.py:92: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Training Epoch: 1:   0%|          | 0/448 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/llama_recipes/utils/train_utils.py:151: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please u

Max CUDA memory allocated was 15 GB
Max CUDA memory reserved was 16 GB
Peak active CUDA memory was 15 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 2 GB
Epoch 1: train_perplexity=1.2751, train_epoch_loss=0.2430, epoch time 5265.642416089773s


### Step 6:
Save model checkpoint

In [10]:
model.save_pretrained(train_config.output_dir)

### Step 7:
Try the fine tuned model on the same example again to see the learning progress:

In [11]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this dialog:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
---
Summary:
스미스씨는 건강검진을 받기 위해 의사에게 방문했습니다. 호킨스 의사는 스미스씨에게 매년 한 번은 오는 것이 좋다고 말했습니다. 스미스씨는 담배를 끊으려고 하지만, 습관을 버리는 것이 어렵다고 말했습니다.


---
---
---
---
---

이 아래를 실행하기 전에 메모리 확보를 위해 커널을 재시작 할 것!

In [1]:
import os
import random
import torch

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    #np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(42)

In [2]:
# https://huggingface.co/blog/peft 이 글에서 학습한걸 읽는 코드를 참고해서 작성함.

import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from llama_recipes.configs import train_config as TRAIN_CONFIG
from peft import PeftModel, PeftConfig

device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
peft_model_id = "meta-llama-samsum"
peftConfig = PeftConfig.from_pretrained(peft_model_id)

from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = LlamaForCausalLM.from_pretrained(
            peftConfig.base_model_name_or_path,
            device_map="auto",
            quantization_config=config,
            use_cache=False,
            torch_dtype=torch.float16,
        )
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained(peftConfig.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
eval_prompt = """
Summarize this dialog:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    generated = model.generate(**model_input, max_new_tokens=100)
    print(tokenizer.decode(generated[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this dialog:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
---
Summary:
호킨스 의사는 스미스씨에게 건강검진을 받으라고 권하고, 그가 담배를 끊으라고 조언합니다.


In [4]:
from torch.utils.data import Dataset
import pandas as pd

class DatasetForLlamaTest(Dataset):
    def __init__(self, train_csv_fullpath):
        df = pd.read_csv(train_csv_fullpath)
        
        prompt = (
            f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n"
        )

        def apply_prompt_template(s):
            return prompt.format(dialog=s)

        df['dialogue'] = df['dialogue'].map(apply_prompt_template)

        self.fname_list = []
        self.preprocessed_list = []
        
        for i in range(len(df)):
            #
            self.fname_list.append(df.iloc[i]['fname'])
            
            # tokenizer() 와 tokenizer.encode() 는 다르다!
            # tokenizer() 를 사용해야 딕셔너리(input_ids, attention_mask) 형태로 리턴됨.
            prompt = tokenizer(df.iloc[i]['dialogue'], return_tensors="pt").to("cuda")
            self.preprocessed_list.append(prompt)
        
        self.len = len(self.preprocessed_list)

    def __getitem__(self, idx):
        return self.fname_list[idx], self.preprocessed_list[idx]

    def __len__(self):
        return self.len   

In [5]:
train_dataset = DatasetForLlamaTest('../../data/test.csv')

In [6]:
import re
from tqdm import tqdm

fname_list = []
summary_list = []

model.eval()
with torch.no_grad():
    for fname, model_input in tqdm(train_dataset):
        generated = model.generate(**model_input, max_new_tokens=100)
        decoded_str = tokenizer.decode(generated[0], skip_special_tokens=True)
        
        summary = decoded_str.split('Summary:\n')[1]
        
        # \n 제거
        summary = re.sub('\n', '', summary)
        
        fname_list.append(fname)
        summary_list.append(summary)

100%|██████████| 499/499 [1:21:00<00:00,  9.74s/it]


In [7]:
print(fname_list)
print(summary_list)

['test_0', 'test_1', 'test_2', 'test_3', 'test_4', 'test_5', 'test_6', 'test_7', 'test_8', 'test_9', 'test_10', 'test_11', 'test_12', 'test_13', 'test_14', 'test_15', 'test_16', 'test_17', 'test_18', 'test_19', 'test_20', 'test_21', 'test_22', 'test_23', 'test_24', 'test_25', 'test_26', 'test_27', 'test_28', 'test_29', 'test_30', 'test_31', 'test_32', 'test_33', 'test_34', 'test_35', 'test_36', 'test_37', 'test_38', 'test_39', 'test_40', 'test_41', 'test_42', 'test_43', 'test_44', 'test_45', 'test_46', 'test_47', 'test_48', 'test_49', 'test_50', 'test_51', 'test_52', 'test_53', 'test_54', 'test_55', 'test_56', 'test_57', 'test_58', 'test_59', 'test_60', 'test_61', 'test_62', 'test_63', 'test_64', 'test_65', 'test_66', 'test_67', 'test_68', 'test_69', 'test_70', 'test_71', 'test_72', 'test_73', 'test_74', 'test_75', 'test_76', 'test_77', 'test_78', 'test_79', 'test_80', 'test_81', 'test_82', 'test_83', 'test_84', 'test_85', 'test_86', 'test_87', 'test_88', 'test_89', 'test_90', 'test_91

In [8]:
data = {
    'fname': fname_list,
    'summary': summary_list,
}

df = pd.DataFrame(data)
df

,fname,summary
0,test_0,#Person1#은 더슨 씨에게 내부 메모를 받아쓰도록 요청합니다. 더슨 씨는 메모...
1,test_1,#Person2#는 교통 체증에 걸려서 늦었다. #Person1#는 #Person2...
2,test_2,#Person1#은 케이트에게 마샤와 히어로가 이혼하려고 한다고 말한다. 케이트는 ...
3,test_3,"#Person1#은 브라이언의 생일을 축하하고, 브라이언은 #Person1#에게 춤..."
4,test_4,#Person2#는 #Person1#에게 올림픽 공원의 중심인 올림픽 스타디움에 대...
...,...,...
494,test_495,잭은 학교 끝나고 자신을 만드는 게임을 하기 위해 찰리를 집에 초대한다. 찰리는 흥...
495,test_496,"#Person2#는 컨트리 음악에 관심을 가졌고, #Person2#는 라디오 방송국..."
496,test_497,#Person1#은 세탁기 사용법을 알고 싶어합니다. 앨리스는 #Person1#에게...
497,test_498,스티브는 매튜에게 최근에 살 곳을 찾고 있다고 말한다. 매튜는 스티브에게 다우 부인...


In [9]:
df.to_csv('../../prediction/output.csv', encoding='utf-8-sig', index=False)